In [60]:
#%pip install openai
#%pip install python-dotenv
%pip install pyjson5
import os
import json
from pprint import pprint, PrettyPrinter
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path
pp = PrettyPrinter(
    indent=2,
    width=1,        # Force maximum line breaks
    depth=None,     # No depth limit
    compact=False,  # Items on separate lines
    sort_dicts=False # Preserve dictionary order
)
load_dotenv()
client = OpenAI(
  organization=os.getenv("OPENAI_ORGANIZATION"),
  project=os.getenv("OPENAI_PROJECT")
)

#current_directory = os.getcwd('../')
#plan_name = "-".join( (parts := current_directory.split('/'))[parts.index("plans"):] )


Note: you may need to restart the kernel to use updated packages.


In [61]:
def build_nested_structure(root_path: str, parent_id: str = None) -> dict:
    """
    Recursively builds a nested object structure from a directory path,
    incorporating index.json contents at each level.
    """
    root = Path(root_path)
    base_name = root.name
    current_id = base_name if parent_id is None else f"{parent_id}-{base_name}"
    
    # Initialize with ID
    structure = {"id": current_id}
    
    # Check for and read index.json
    index_path = root / "index.json"
    if index_path.exists():
        try:
            with open(index_path) as f:
                index_data = json.load(f)
                structure.update(index_data)
        except (json.JSONDecodeError, Exception) as e:
            print(f"Warning: Error with index.json in {root_path}: {str(e)}")
    
    # Get subdirectories
    subdirs = [d for d in root.iterdir() if d.is_dir()]
    if subdirs:
        structure["pods"] = {"items": []}
        for subdir in subdirs:
            sub_structure = build_nested_structure(str(subdir), current_id)
            structure["pods"]["items"].append(sub_structure)
            
    return structure

In [63]:
if __name__ == "__main__":
    result = build_nested_structure("plans")
    print(json.dumps(result, indent=2))

{
  "id": "plans",
  "pods": {
    "items": [
      {
        "id": "plans-plan"
      },
      {
        "id": "plans-agent",
        "pods": {
          "items": [
            {
              "id": "plans-agent-product_owner",
              "pods": {
                "items": [
                  {
                    "id": "plans-agent-product_owner-scrum_master",
                    "pods": {
                      "items": [
                        {
                          "id": "plans-agent-product_owner-scrum_master-developer"
                        },
                        {
                          "id": "plans-agent-product_owner-scrum_master-researcher",
                          "desc": "You are the Product Owner, a key role in the Scrum framework responsible for maximizing the value of the product. Your primary focus is on defining and prioritizing work based on business goals and customer needs. You act as the bridge between stakeholders, the development team, and the

In [59]:
def initialize_thread(client: OpenAI, name: Optional[str] = None) -> str:
    try:
        # Create new thread with metadata
        thread = client.beta.threads.create(
            metadata={
                "name": name
            } if name else None
        )
        return thread.id
        
    except Exception as e:
        raise Exception(f"Failed to initialize thread: {str(e)}")

In [45]:
thread_id = initialize_thread(client, plan_name)

NameError: name 'plan_name' is not defined

In [20]:
print(thread_id)

thread_ItiqszTHSeEGmsFtlB3zRycf


In [10]:
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

This is a test. How can I assist you today?